In [ ]:
#r "BoSSSpad.dll"
//#r "D:\BoSSS2\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\bossspad.dll"

using System;
using System.Collections.Generic;
using System.Linq;
using System.IO;
using System.Data;
using System.Globalization;
using System.Threading;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Platform.LinAlg;
using BoSSS.Solution.NSECommon;
using BoSSS.Application.XNSEC;
Init();

In [ ]:
BoSSSshell.WorkflowMgm.Init("HeatedCavity_NusseltStudy");

In [ ]:
var myBatch = BoSSSshell.GetDefaultQueue();
var myDb = myBatch.CreateOrOpenCompatibleDatabase("HeatedCavity_NusseltStudy");



In [ ]:

Dictionary<double,double> NusseltSolutions = new Dictionary<double,double>();
NusseltSolutions.Add(1e2, 0.9787);
NusseltSolutions.Add(1e3, 1.1077);
NusseltSolutions.Add(1e4, 2.2180);
NusseltSolutions.Add(1e5, 4.4800);
NusseltSolutions.Add(1e6, 8.6870);
NusseltSolutions.Add(1e7, 16.2400);

In [ ]:

Plot2Ddata[,] PlotTable = new Plot2Ddata[2, 3];
for (int iCol = 0; iCol < 2; iCol++) {
    for (int iRow = 0; iRow < 3; iRow++) {
        double[] RayleighS = new double[]{1e2,1e6};
        int[] dgDegrees = new int[]{2,3,4};

        var AllSessionsForRa = myDb.Sessions.Where(sess=>Convert.ToDouble(sess.KeysAndQueries["Rayleigh"]) == RayleighS[iCol]);

        var sessionsforDg   =  AllSessionsForRa.Where(sess => sess.Name.Contains("DG"+dgDegrees[iRow])).OrderBy(sess => sess.GetGrids().Pick(0).NumberOfCells);
        double[] NoOfCells = new double[sessionsforDg.Count()];
        double[] LeftNusseltS = new double[sessionsforDg.Count()];
        double[] RightNusseltS = new double[sessionsforDg.Count()];
        int cnt = 0;
        foreach(var sess in sessionsforDg ){
            NoOfCells[cnt] = Math.Sqrt(sess.GetGrids().Pick(0).NumberOfCells);
            LeftNusseltS[cnt] = Convert.ToDouble(sess.KeysAndQueries["NusseltNumber0"]) / 1.2;
            RightNusseltS[cnt]  = Convert.ToDouble(sess.KeysAndQueries["NusseltNumber1"]) / 1.2*(-1);
            cnt++;    
        }


        LineColors[]  allColors = Enum.GetValues(typeof(LineColors)).Cast<LineColors>().ToArray();
        PointTypes[] myPointTypes = new PointTypes[]{ PointTypes.Diamond, PointTypes.Box,  PointTypes.LowerTriangle,PointTypes.OpenLowerTriangle, };
        var plot = new Plot2Ddata();
        var fmt = new PlotFormat();
        fmt.Style = Styles.LinesPoints;
        fmt.PointType =  PointTypes.Diamond;
        fmt.LineColor = LineColors.Blue;
        plot.AddDataGroup( "Nu_h",NoOfCells, LeftNusseltS, fmt);

        var fmt2 = new PlotFormat();
        fmt2.Style = Styles.LinesPoints;
        fmt2.DashType = DashTypes.Solid;
        fmt2.LineColor = LineColors.Green;
        plot.AddDataGroup( "Nu_c",NoOfCells, RightNusseltS, fmt2);

        double Ra = iCol == 0? Ra = 1e2: 1e6;
        double[] xRef = new double[]{0.0,140};
        double[] yRef = new double[]{NusseltSolutions[Ra],NusseltSolutions[Ra]};
        
        var fmt3 = new PlotFormat();
        fmt3.Style = Styles.Lines;
        fmt3.DashType = DashTypes.DotDashed;
        fmt3.LineWidth = 2;
        fmt3.LineColor = LineColors.Red;
        plot.AddDataGroup( "Reference Value",xRef, yRef, fmt3);
        
        string mytitle = "";

        if(iCol == 0 && iRow == 0){
            mytitle = "Ra= "+ Ra + ",dg= " + 2; 
        } else if(iCol == 0 && iRow == 1){
            mytitle = "Ra= "+ Ra + ",dg= " + 3; 
        }else if(iCol == 0 && iRow == 2){
            mytitle = "Ra= "+ Ra + ",dg= " + 4; 
        }else if(iCol == 1 && iRow == 0){
            mytitle = "Ra= "+ Ra + ",dg= " + 2; 
        }else if(iCol == 1 && iRow == 1){
            mytitle = "Ra= "+ Ra + ",dg= " + 3; 
        }else if(iCol == 1 && iRow == 2){
            mytitle = "Ra= "+ Ra + ",dg= " + 4; 
        } else {throw new Exception("");}
        plot.Title = mytitle;
        plot.LegendPosition = new double[]{130,0.97};
        PlotTable[iCol,iRow] = plot;   




    }
}


var gp = PlotTable.ToGnuplot();
gp.PlotSVG(xRes:1400,yRes:800)

